# Collecting downloaded files and Save as a parquet file

In [29]:
import polars as pl
import os

def is_not_valid(f: str) -> bool:
    return not f.endswith('.csv') or '-' in f 
    # or '.P,' not in f

In [30]:
def convert_k(val):
    if isinstance(val, str) and val.endswith('K'):
        return float(val[:-1]) * 1000
    return float(val)

def collect(folder):
    ohlcv = []
    for f in os.listdir(folder):
        if is_not_valid(f):
            continue

        path = os.path.join(folder, f)
        try:
            df = pl.read_csv(
                path, 
                schema_overrides={c: pl.Float64 for c in ['open', 'high', 'low', 'close', 'Volume']},
                null_values=["∅"], ignore_errors=True
            )
        except Exception as e:
            print(f"❌ Skipping file {f} due to read error: {e}")
            continue
        
        if 'date' in df.columns: # ['date', 'time', 'open', 'high', 'low', 'close', 'vol']
            df = df.with_columns(
                (pl.col("date") + " " + pl.col("time")).str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M").alias("datetime")
            )
            df = df.with_columns(
                (pl.col("datetime").cast(pl.Int64))# nanoseconds → seconds
                .alias("time")
            )
            df = df.select(["time", "open", "high", "low", "close", pl.col("vol").alias("Volume")])
            if df.schema.get('Volume') == pl.String:
                df = df.with_columns(
                    pl.col("Volume")
                    .str.replace_all(r"([0-9.]+)K", r"${1}000")
                    .str.replace_all(r"([0-9.]+)M", r"${1}000000")
                    .str.strip_chars()
                    .cast(pl.Float64)
                    .alias("Volume")
                )
        if df.schema.get('time') != pl.Int64:
            continue

        f = f.replace(', 1D.csv', ', 1440.csv')  # Fix for some files with space
        df = df.with_columns([
            pl.lit(f.split('_')[-2].lower()).alias('Exchange'),
            pl.lit(f.split('_')[-1].split(',')[0]).alias('Pair'),
            pl.lit(f.split(',')[-1].split('.')[0].strip().split()[0]).cast(pl.Int16).alias('Interval'),
            pl.lit(f).alias('FileName'),
        ])
 
        
        ohlcv.append(df)
    print(f"✅ Processed {len(ohlcv)} files from {folder}")
    return ohlcv

In [31]:

ohlcv_dumdum = collect('../../dumdum')
ohlcv_root = collect('../../')
ohlcv_root = []
df_all = pl.concat(ohlcv_dumdum + ohlcv_root, how="vertical").unique()

✅ Processed 8379 files from ../../dumdum
✅ Processed 1847 files from ../../


In [32]:
print(
    f"✅ Total {len(df_all)} unique rows from all files. Columns: {df_all.columns}"
)
df_all.schema

✅ Total 95871172 unique rows from all files. Columns: ['time', 'open', 'high', 'low', 'close', 'Volume', 'Exchange', 'Pair', 'Interval', 'FileName']


Schema([('time', Int64),
        ('open', Float64),
        ('high', Float64),
        ('low', Float64),
        ('close', Float64),
        ('Volume', Float64),
        ('Exchange', String),
        ('Pair', String),
        ('Interval', Int16),
        ('FileName', String)])

In [33]:

# 병합 및 중복 제거
prev = pl.read_parquet('../../tradingview_ohlcv_old.parquet').with_columns(
    (pl.col("time").cast(pl.Int64) // 1_000_000_000).alias("time"),
    (pl.lit(9999).cast(pl.Int16)).alias('Interval'),
    (pl.lit('first-version')).alias('FileName'),
)
print(f"✅ Loaded previous data with {prev.shape[0]} rows")
df_all = pl.concat([prev, df_all], how="vertical").unique()



df_all = df_all.with_columns(
    (pl.col("Pair").str.contains(r"\.(P|PS)$")).alias("Perpetual")
)
df_all = df_all.filter((pl.col("time").is_not_null()) & (pl.col('Volume').is_not_null()) & (pl.col('close').is_not_null()))


df_all = (
    df_all.sort(["FileName"])  # 우선 C 오름차순 정렬
      .unique(subset=["time",'Exchange', 'Pair', 'Interval'], keep="first")  # A, B 기준으로 유니크하게, C가 작은 row 유지
) #  "open", 'high', 'low', 'Volume',  

df_all

✅ Loaded previous data with 12613878 rows


time,open,high,low,close,Volume,Exchange,Pair,Interval,FileName,Perpetual
i64,f64,f64,f64,f64,f64,str,str,i16,str,bool
1689256800,30673.4,30697.7,30474.1,30570.8,534321.14526,"""okx""","""BTCUSDC.P""",60,"""20250705_213609_OKX_BTCUSDC.P,…",true
1740863700,604.84,605.33,604.39,604.81,5.67,"""bitmart""","""BNBUSDT""",15,"""20250708_235845_BITMART_BNBUSD…",false
1751923800,2.8331,2.85,2.8323,2.8426,191448.0,"""binance""","""SUIUSDT""",15,"""20250708_221228_BINANCE_SUIUSD…",false
1701118800,13.95,14.128,13.935,14.062,96952.47,"""phemex""","""LINKUSDT.P""",60,"""20250706_043745_PHEMEX_LINKUSD…",true
1581379200,222.993539,239.154429,218.2,236.737547,51495.275809,"""poloniex""","""ETHUSDT""",1440,"""20250708_141527_POLONIEX_ETHUS…",false
…,…,…,…,…,…,…,…,…,…,…
1679248800,1811.84,1824.93,1808.46,1824.54,27226.0,"""capitalcom""","""ETHUSD""",9999,"""first-version""",false
1701597600,39472.6,39556.4,39428.1,39553.3,104.12,"""cryptocom""","""BTCUSD.P""",60,"""20250705_215033_CRYPTOCOM_BTCU…",true
1735974000,7.781,7.795,7.745,7.753,287265.9,"""coinw""","""DOTUSDT.P""",60,"""20250706_021102_COINW_DOTUSDT.…",true


In [34]:
# 1. 중복되는 조합의 키 만들기
df = df_all.with_columns([
    (pl.col("time").cast(pl.String) + "|" + pl.col("Exchange") + "|" + pl.col("Pair")+ "|" + pl.col("Interval").cast(pl.String)).alias("key")
])

# 2. 중복 key를 가진 row만 필터링
duplicated_keys = (
    df.group_by("key")
    .agg(pl.len().alias("cnt"))
    .filter(pl.col("cnt") > 1)
    .select("key")
)

df_dupes = df.join(duplicated_keys, on="key", how="inner").drop("key").sort(['time', 'Exchange', 'Pair'])
df_dupes

time,open,high,low,close,Volume,Exchange,Pair,Interval,FileName,Perpetual
i64,f64,f64,f64,f64,f64,str,str,i16,str,bool


In [35]:
df_all.write_parquet('../tradingview_ohlcv.parquet')

# Statistical Description

In [2]:
import polars as pl

df_all = pl.read_parquet('../tradingview_ohlcv.parquet')

In [53]:
df_spot = df_all.filter(~pl.col("Perpetual")).to_pandas()
df_perp = df_all.filter(pl.col("Perpetual")).to_pandas()

In [58]:
import pandas as pd

df_spot['time'] = pd.to_datetime(df_spot['time'], unit='s', errors='coerce')
df_perp['time'] = pd.to_datetime(df_perp['time'], unit='s', errors='coerce')
print(df_spot.shape, df_perp.shape)
print(df_spot.columns)


df_spot_binance = df_spot[df_spot['Exchange'] == 'binance'].copy()
df_spot_binance.value_counts('Interval', dropna=False).sort_index()

(48637216, 11) (13066379, 11)
Index(['time', 'open', 'high', 'low', 'close', 'Volume', 'Exchange', 'Pair',
       'Interval', 'FileName', 'Perpetual'],
      dtype='object')


Interval
1        301156
15      1203184
20       410008
30        27268
60       400692
1440     108722
9999     686987
Name: count, dtype: int64

In [60]:
df_spot.describe(include='all')

,time,open,high,low,close,Volume,Exchange,Pair,Interval,FileName,Perpetual
count,48603439,4.863722e+07,4.863722e+07,4.863722e+07,4.863722e+07,4.804751e+07,48637216,48637216,4.863722e+07,48637216,48637216
unique,NaN,NaN,NaN,NaN,NaN,NaN,39,115,NaN,4654,1
top,NaN,NaN,NaN,NaN,NaN,NaN,binance,BTCUSDT,NaN,first-version,False
freq,NaN,NaN,NaN,NaN,NaN,NaN,3138017,3503107,NaN,10335321,48637216
mean,2024-10-10 09:41:21.078161664,5.052994e+05,5.062661e+05,5.043206e+05,5.053146e+05,2.653273e+09,NaN,NaN,2.184988e+03,NaN,NaN
min,2011-08-18 00:00:00,0.000000e+00,1.500000e-07,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,1.000000e+00,NaN,NaN
25%,2024-09-07 07:00:00,2.251600e+00,2.257440e+00,2.245600e+00,2.251600e+00,5.247568e+00,NaN,NaN,1.500000e+01,NaN,NaN
50%,2025-02-02 10:15:00,1.571098e+02,1.577955e+02,1.565200e+02,1.571000e+02,3.378236e+02,NaN,NaN,2.000000e+01,NaN,NaN
75%,2025-05-07 17:00:00,3.366600e+03,3.378400e+03,3.354200e+03,3.366570e+03,1.263204e+04,NaN,NaN,6.000000e+01,NaN,NaN
max,2025-07-10 03:59:00,1.624540e+08,1.634600e+08,1.618240e+08,1.624700e+08,2.542703e+14,NaN,NaN,9.999000e+03,NaN,NaN


In [61]:
df_perp.describe(include='all')

,time,open,high,low,close,Volume,Exchange,Pair,Interval,FileName,Perpetual
count,13066379,1.306638e+07,1.306638e+07,1.306638e+07,1.306638e+07,1.306570e+07,13066379,13066379,1.306638e+07,13066379,13066379
unique,NaN,NaN,NaN,NaN,NaN,NaN,27,104,NaN,1605,1
top,NaN,NaN,NaN,NaN,NaN,NaN,bingx,BTCUSDT.P,NaN,first-version,True
freq,NaN,NaN,NaN,NaN,NaN,NaN,1457118,654828,NaN,178659,13066379
mean,2024-06-09 08:07:13.740308992,6.970964e+03,6.991591e+03,6.950086e+03,6.971334e+03,6.954582e+09,NaN,NaN,2.063235e+02,NaN,NaN
min,2019-11-18 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,1.000000e+00,NaN,NaN
25%,2023-11-12 06:00:00,6.337000e-01,6.380000e-01,6.300000e-01,6.337000e-01,4.837000e+03,NaN,NaN,6.000000e+01,NaN,NaN
50%,2024-07-20 05:00:00,1.146800e+01,1.155200e+01,1.138900e+01,1.146900e+01,7.806785e+04,NaN,NaN,6.000000e+01,NaN,NaN
75%,2025-02-03 05:00:00,2.455000e+02,2.468500e+02,2.443228e+02,2.455000e+02,1.215332e+06,NaN,NaN,6.000000e+01,NaN,NaN
max,2025-07-10 03:40:00,1.120107e+05,1.120872e+05,1.117870e+05,1.120107e+05,2.312292e+15,NaN,NaN,9.999000e+03,NaN,NaN


In [62]:
df_spot.time.value_counts().to_frame().sort_index()

,count
time,
2011-08-18 00:00:00,1
2011-08-19 00:00:00,1
2011-08-20 00:00:00,1
2011-08-21 00:00:00,1
2011-08-22 00:00:00,1
...,...
2025-07-10 03:55:00,7
2025-07-10 03:56:00,7
2025-07-10 03:57:00,4


In [63]:
df_spot.Exchange.value_counts().to_frame().sort_index()

,count
Exchange,
binance,3138017
binanceus,1896551
bingx,281
bitfinex,850502
bitflyer,147788
bitget,1744087
bithumb,834422
bitmart,1710678
bitmex,373334


In [64]:
df_spot.Pair.value_counts().to_frame().head(20)

,count
Pair,
BTCUSDT,3503107
ETHUSDT,3305000
XRPUSDT,3223357
ETHUSDC,2418588
BTCUSDC,2330853
SOLUSDT,2256926
ETHUSD,1955866
XRPUSDC,1815078
BTCUSD,1696439


In [65]:
df_spot.Pair.value_counts().to_frame().tail(20)

,count
Pair,
MATICUSDC,22731
1000BONKUSDT,22062
BNBKRW,21704
USTUSDT,20118
SOLGUSD,10957
DOGEGUSD,8942
LTCGUSD,8918
PEPEGUSD,8908
SHIBGUSD,8649


In [66]:
df_perp.time.value_counts().to_frame().sort_index()

,count
time,
2019-11-18 00:00:00,1
2019-11-19 00:00:00,1
2019-11-20 00:00:00,1
2019-11-21 00:00:00,1
2019-11-22 00:00:00,1
...,...
2025-07-10 03:36:00,1
2025-07-10 03:37:00,1
2025-07-10 03:38:00,1


In [67]:
df_perp.Exchange.value_counts().to_frame().sort_index()

,count
Exchange,
binance,980361
bingx,1457118
bitget,798518
bitmart,477020
bitmex,687818
blofin,429142
bybit,849066
coinbase,278712
coinw,128711


In [68]:
df_perp.Pair.value_counts().to_frame().head(20)

,count
Pair,
BTCUSDT.P,654828
DOTUSDT.P,444385
ADAUSDT.P,438174
ETHUSDT.P,428893
DOGEUSDT.P,424611
AVAXUSDT.P,424455
SOLUSDT.P,424132
BCHUSDT.P,421669
XRPUSDT.P,419966


In [69]:
df_perp.Pair.value_counts().to_frame().tail(20)

,count
Pair,
BTCGUSD.P,15826
ETHGUSD.P,14936
PEPEGUSD.P,11975
SOLGUSD.P,11293
XRPGUSD.P,9842
DOGEGUSD.P,9738
AVAXGUSD.P,9037
SHIBUSDC.P,8975
LTCGUSD.P,8284
